# IBM Capstone project

__Week 5 Report__

Link: https://github.com/GokulParzival/Detecting-the-best-waterfalls-spot-in-Tamilnadu-to-set-up-a-restaurant/edit/main/Detecting%20the%20best%20waterfalls%20spot%20in%20Tamilnadu%20to%20set%20up%20a%20restaurant.ipynb

## Aim

##### To locate the best waterfalls spot in Tamilnadu to set up a restaurant

## Task Done

1. Building a dataframe from the data collected from scrapping the Waterfalls wikipedia Page.
2. Obtaining the geographical locations of the waterfalls
3. Obtaining the venues around the waterfalls
4. Obtaining the data of the eateries from the venue.
5. Suggesting  best waterfalls spot in Tamilnadu to set up a restaurant.

## Code

### Importing necessary Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder# convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

print('Folium installed')
print('Libraries imported.')

Libraries imported.
Folium installed
Libraries imported.


In [2]:
from bs4 import BeautifulSoup

### Data Collection

In [3]:
data = requests.get('https://en.wikipedia.org/wiki/Category:Waterfalls_of_Tamil_Nadu').text

In [4]:
soup = BeautifulSoup(data, 'html.parser')

In [5]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Category:Waterfalls of Tamil Nadu - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"1344a598-0306-405d-83e3-f648cca2a84d","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Waterfalls_of_Tamil_Nadu","wgTitle":"Waterfalls of Tamil Nadu","wgCurRevisionId":827275395,"wgRevisionId":827275395,"wgArticleId":22080609,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Commons category link is on Wikidata","Waterfalls of India by state or union territory","L

In [6]:
name = []

In [7]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    name.append(row.text)

In [8]:
waterfall_df = pd.DataFrame({"Name": name,})
waterfall_df

,Name
0,Agasthiyar Falls
1,Agaya Gangai
2,Aintharuvi
3,Ayyanar Falls
4,Catherine Falls
5,Chinnakallar Falls
6,Courtallam
7,Coutrallam Falls
8,Gaur Vellaiyan Falls
9,Hogenakkal Eco Tourism


The final data obtained

In [9]:
waterfall_df.shape

(28, 1)

### Obtaining the geographical coordinates

In [10]:
def latlng(place):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, TamilNadu, India'.format(place))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
coords = [ latlng(place) for place in waterfall_df["Name"].tolist() ]

In [12]:
coords

[[8.704270000000065, 77.36353000000008],
 [11.170130000000029, 78.39562000000006],
 [8.674980000000062, 77.68513000000007],
 [10.836920000000077, 78.59468000000004],
 [11.433240000000069, 76.87888000000004],
 [11.680440000000033, 77.76561000000004],
 [8.932710000000043, 77.27252000000004],
 [8.927910000000054, 77.27851000000004],
 [11.480730000000051, 78.77598000000006],
 [12.121800000000064, 77.77807000000007],
 [12.121800000000064, 77.77807000000007],
 [11.322690000000023, 76.73040000000003],
 [11.758260000000064, 78.22011000000003],
 [8.658250000000066, 77.31949000000003],
 [10.18363000000005, 77.53056000000004],
 [10.115920000000074, 77.99417000000005],
 [11.35256000000004, 76.78539000000006],
 [10.457710000000077, 76.97097000000008],
 [11.450000000000045, 76.58333000000005],
 [9.238300000000038, 77.68276000000003],
 [10.943840000000023, 76.72132000000005],
 [9.661180000000058, 77.30500000000006],
 [10.22730000000007, 77.55489000000006],
 [8.394870000000026, 77.26283000000006],
 [8

In [13]:
coordinates = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [14]:
waterfall_df['Latitude'] = coordinates['Latitude']
waterfall_df['Longitude'] = coordinates['Longitude']

In [15]:
waterfall_df

,Name,Latitude,Longitude
0,Agasthiyar Falls,8.704270,77.363530
1,Agaya Gangai,11.170130,78.395620
2,Aintharuvi,8.674980,77.685130
3,Ayyanar Falls,10.836920,78.594680
4,Catherine Falls,11.433240,76.878880
5,Chinnakallar Falls,11.680440,77.765610
6,Courtallam,8.932710,77.272520
7,Coutrallam Falls,8.927910,77.278510
8,Gaur Vellaiyan Falls,11.480730,78.775980
9,Hogenakkal Eco Tourism,12.121800,77.778070


In [16]:
waterfall_df.to_csv("waterfall_df.csv", index=False)

### Creating a map

In [17]:
address = 'Tamil Nadu, India'

geolocator = Nominatim(user_agent="tn_apps")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tamilnadu, India is  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tamilnadu, India is  10.9094334, 78.3665347.


In [18]:
# create map of New York using latitude and longitude values
map_tn = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(waterfall_df['Latitude'], waterfall_df['Longitude'], waterfall_df['Name']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_tn)  
    
map_tn

In [19]:
map_tn.save('map_tn.html')

### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'xxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxxxxx' # your Foursquare Secret
VERSION = 'xxxxxxxxxxxx' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Searching for 50 facilities within the radius of 1000 m

In [47]:
radius = 1000
LIMIT = 50

venues = []

for lat, long, neighborhood in zip(waterfall_df['Latitude'], waterfall_df['Longitude'], waterfall_df['Name']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [48]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Name', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(26, 7)


,Name,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Catherine Falls,11.43324,76.87888,Longwood Shola Reserved Forest,11.427961,76.872499,Trail
1,Courtallam,8.93271,77.27252,Coutralam Main Falls,8.932842,77.269854,Scenic Lookout
2,Courtallam,8.93271,77.27252,Courtallam,8.934178,77.274584,Scenic Lookout
3,Courtallam,8.93271,77.27252,Saaral resort,8.935619,77.269792,Resort
4,Courtallam,8.93271,77.27252,Courtalam Five Falls,8.933659,77.264090,Scenic Lookout


__No of Venues in each Waterfall__

In [49]:
venues_df.groupby(["Name"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Name,,,,,,
Catherine Falls,1,1,1,1,1,1
Courtallam,4,4,4,4,4,4
Coutrallam Falls,5,5,5,5,5,5
Hogenakkal Eco Tourism,3,3,3,3,3,3
Hogenakkal Falls,3,3,3,3,3,3
"Law's Falls, Coonoor",3,3,3,3,3,3
Siruvani Waterfalls,1,1,1,1,1,1
Thirparappu,2,2,2,2,2,2
Tirparappu Waterfalls,2,2,2,2,2,2


In [50]:
venues_df['VenueCategory'].unique()

array(['Trail', 'Scenic Lookout', 'Resort', 'Bubble Tea Shop',
       'Bus Station', 'Indian Restaurant', 'Hotel', 'Train Station',
       'Asian Restaurant', 'Forest', 'Other Great Outdoors',
       'Coffee Shop'], dtype=object)

### Analysis of each Waterfall Venue

In [51]:
# one hot encoding
wf_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
wf_onehot['Name'] = venues_df['Name'] 

# move neighborhood column to the first column
fixed_columns = [wf_onehot.columns[-1]] + list(wf_onehot.columns[:-1])
wf_onehot = wf_onehot[fixed_columns]

print(wf_onehot.shape)
wf_onehot.head()

(26, 13)


,Name,Asian Restaurant,Bubble Tea Shop,Bus Station,Coffee Shop,Forest,Hotel,Indian Restaurant,Other Great Outdoors,Resort,Scenic Lookout,Trail,Train Station
0,Catherine Falls,0,0,0,0,0,0,0,0,0,0,1,0
1,Courtallam,0,0,0,0,0,0,0,0,0,1,0,0
2,Courtallam,0,0,0,0,0,0,0,0,0,1,0,0
3,Courtallam,0,0,0,0,0,0,0,0,1,0,0,0
4,Courtallam,0,0,0,0,0,0,0,0,0,1,0,0


In [84]:
wf_grouped = wf_onehot.groupby(["Name"]).mean().reset_index()

print(wf_grouped.shape)
wf_grouped

(10, 13)


,Name,Asian Restaurant,Bubble Tea Shop,Bus Station,Coffee Shop,Forest,Hotel,Indian Restaurant,Other Great Outdoors,Resort,Scenic Lookout,Trail,Train Station
0,Catherine Falls,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,1.0,0.000000
1,Courtallam,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.25,0.750000,0.0,0.000000
2,Coutrallam Falls,0.000000,0.2,0.2,0.0,0.0,0.000000,0.200000,0.0,0.00,0.400000,0.0,0.000000
3,Hogenakkal Eco Tourism,0.000000,0.0,0.0,0.0,0.0,0.666667,0.000000,0.0,0.00,0.333333,0.0,0.000000
4,Hogenakkal Falls,0.000000,0.0,0.0,0.0,0.0,0.666667,0.000000,0.0,0.00,0.333333,0.0,0.000000
5,"Law's Falls, Coonoor",0.333333,0.0,0.0,0.0,0.0,0.000000,0.333333,0.0,0.00,0.000000,0.0,0.333333
6,Siruvani Waterfalls,0.000000,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.000000
7,Thirparappu,0.000000,0.0,0.5,0.0,0.0,0.000000,0.000000,0.5,0.00,0.000000,0.0,0.000000
8,Tirparappu Waterfalls,0.000000,0.0,0.5,0.0,0.0,0.000000,0.000000,0.5,0.00,0.000000,0.0,0.000000
9,Vaideki Falls,0.000000,0.0,0.0,0.5,0.5,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.000000


In [85]:
wf_grouped.dtypes

Name                     object
Asian Restaurant        float64
Bubble Tea Shop         float64
Bus Station             float64
Coffee Shop             float64
Forest                  float64
Hotel                   float64
Indian Restaurant       float64
Other Great Outdoors    float64
Resort                  float64
Scenic Lookout          float64
Trail                   float64
Train Station           float64
dtype: object

In [86]:
len(wf_grouped[wf_grouped[["Asian Restaurant","Bubble Tea Shop","Hotel","Indian Restaurant"]] > 0])

10

### The falls containing eateries

In [87]:
wf_eat = wf_grouped[["Name","Asian Restaurant","Bubble Tea Shop","Hotel","Indian Restaurant"]]

In [88]:
wf_eat

,Name,Asian Restaurant,Bubble Tea Shop,Hotel,Indian Restaurant
0,Catherine Falls,0.000000,0.0,0.000000,0.000000
1,Courtallam,0.000000,0.0,0.000000,0.000000
2,Coutrallam Falls,0.000000,0.2,0.000000,0.200000
3,Hogenakkal Eco Tourism,0.000000,0.0,0.666667,0.000000
4,Hogenakkal Falls,0.000000,0.0,0.666667,0.000000
5,"Law's Falls, Coonoor",0.333333,0.0,0.000000,0.333333
6,Siruvani Waterfalls,0.000000,0.0,0.000000,0.000000
7,Thirparappu,0.000000,0.0,0.000000,0.000000
8,Tirparappu Waterfalls,0.000000,0.0,0.000000,0.000000
9,Vaideki Falls,0.000000,0.0,0.000000,0.000000


### Clustering the Waterfall

In [71]:
# set number of clusters
kclusters = 3

wf_clustering = wf_grouped.drop(["Name"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(wf_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 0, 0, 1, 2, 1, 1, 2])

In [72]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
wf_merged = wf_eat.copy()

# add clustering labels
wf_merged["Cluster Labels"] = kmeans.labels_

In [73]:
wf_merged.rename(columns={"Name": "Name"}, inplace=True)
wf_merged.head()

,Name,Asian Restaurant,Bubble Tea Shop,Coffee Shop,Hotel,Indian Restaurant,Cluster Labels
0,Catherine Falls,0.0,0.0,0.0,0.000000,0.0,1
1,Courtallam,0.0,0.0,0.0,0.000000,0.0,0
2,Coutrallam Falls,0.0,0.2,0.0,0.000000,0.2,0
3,Hogenakkal Eco Tourism,0.0,0.0,0.0,0.666667,0.0,0
4,Hogenakkal Falls,0.0,0.0,0.0,0.666667,0.0,0


In [74]:
wf_merged = wf_merged.join(waterfall_df.set_index("Name"), on="Name")

print(wf_merged.shape)
wf_merged.head()

(10, 9)


,Name,Asian Restaurant,Bubble Tea Shop,Coffee Shop,Hotel,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,Catherine Falls,0.0,0.0,0.0,0.000000,0.0,1,11.43324,76.87888
1,Courtallam,0.0,0.0,0.0,0.000000,0.0,0,8.93271,77.27252
2,Coutrallam Falls,0.0,0.2,0.0,0.000000,0.2,0,8.92791,77.27851
3,Hogenakkal Eco Tourism,0.0,0.0,0.0,0.666667,0.0,0,12.12180,77.77807
4,Hogenakkal Falls,0.0,0.0,0.0,0.666667,0.0,0,12.12180,77.77807


In [75]:
# sort the results by Cluster Labels
print(wf_merged.shape)
wf_merged.sort_values(["Cluster Labels"], inplace=True)
wf_merged

(10, 9)


,Name,Asian Restaurant,Bubble Tea Shop,Coffee Shop,Hotel,Indian Restaurant,Cluster Labels,Latitude,Longitude
1,Courtallam,0.000000,0.0,0.0,0.000000,0.000000,0,8.932710,77.272520
2,Coutrallam Falls,0.000000,0.2,0.0,0.000000,0.200000,0,8.927910,77.278510
3,Hogenakkal Eco Tourism,0.000000,0.0,0.0,0.666667,0.000000,0,12.121800,77.778070
4,Hogenakkal Falls,0.000000,0.0,0.0,0.666667,0.000000,0,12.121800,77.778070
0,Catherine Falls,0.000000,0.0,0.0,0.000000,0.000000,1,11.433240,76.878880
5,"Law's Falls, Coonoor",0.333333,0.0,0.0,0.000000,0.333333,1,11.352560,76.785390
7,Thirparappu,0.000000,0.0,0.0,0.000000,0.000000,1,8.394870,77.262830
8,Tirparappu Waterfalls,0.000000,0.0,0.0,0.000000,0.000000,1,8.391590,77.259410
6,Siruvani Waterfalls,0.000000,0.0,0.0,0.000000,0.000000,2,10.943840,76.721320
9,Vaideki Falls,0.000000,0.0,0.5,0.000000,0.000000,2,12.863642,80.105244


### Visualizing the clusters

In [76]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(wf_merged['Latitude'], wf_merged['Longitude'], wf_merged['Name'], wf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [77]:
map_clusters.save('map_clusters.html')

### Examining the clusters

__Cluster 1__

In [78]:
wf_merged.loc[wf_merged['Cluster Labels'] == 0]

,Name,Asian Restaurant,Bubble Tea Shop,Coffee Shop,Hotel,Indian Restaurant,Cluster Labels,Latitude,Longitude
1,Courtallam,0.0,0.0,0.0,0.000000,0.0,0,8.93271,77.27252
2,Coutrallam Falls,0.0,0.2,0.0,0.000000,0.2,0,8.92791,77.27851
3,Hogenakkal Eco Tourism,0.0,0.0,0.0,0.666667,0.0,0,12.12180,77.77807
4,Hogenakkal Falls,0.0,0.0,0.0,0.666667,0.0,0,12.12180,77.77807


__Cluster 2__

In [79]:
wf_merged.loc[wf_merged['Cluster Labels'] == 1]

,Name,Asian Restaurant,Bubble Tea Shop,Coffee Shop,Hotel,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,Catherine Falls,0.000000,0.0,0.0,0.0,0.000000,1,11.43324,76.87888
5,"Law's Falls, Coonoor",0.333333,0.0,0.0,0.0,0.333333,1,11.35256,76.78539
7,Thirparappu,0.000000,0.0,0.0,0.0,0.000000,1,8.39487,77.26283
8,Tirparappu Waterfalls,0.000000,0.0,0.0,0.0,0.000000,1,8.39159,77.25941


__Cluster 3__

In [80]:
wf_merged.loc[wf_merged['Cluster Labels'] == 2]

,Name,Asian Restaurant,Bubble Tea Shop,Coffee Shop,Hotel,Indian Restaurant,Cluster Labels,Latitude,Longitude
6,Siruvani Waterfalls,0.0,0.0,0.0,0.0,0.0,2,10.943840,76.721320
9,Vaideki Falls,0.0,0.0,0.5,0.0,0.0,2,12.863642,80.105244


### Inference from Cluster 3

In [81]:
C1=wf_merged.loc[wf_merged['Cluster Labels'] == 2]

In [89]:
C1_res = C1[["Asian Restaurant","Hotel","Indian Restaurant"]]==0.0
C1_res

,Asian Restaurant,Hotel,Indian Restaurant
6,True,True,True
9,True,True,True


In [90]:
C_r = C1_res["Asian Restaurant"]&C1_res["Hotel"]&C1_res["Indian Restaurant"]
C_r

6    True
9    True
dtype: bool

### Inference from Cluster 2

In [93]:
C2=wf_merged.loc[wf_merged['Cluster Labels'] == 1]

In [94]:
C2_res = C2[["Asian Restaurant","Hotel","Indian Restaurant"]]==0.0
C2_res

,Asian Restaurant,Hotel,Indian Restaurant
0,True,True,True
5,False,True,False
7,True,True,True
8,True,True,True


In [95]:
C_r = C2_res["Asian Restaurant"]&C2_res["Hotel"]&C2_res["Indian Restaurant"]
C_r

0     True
5    False
7     True
8     True
dtype: bool

### Inference from Cluster 1

In [96]:
C3=wf_merged.loc[wf_merged['Cluster Labels'] == 0]

In [97]:
C3_res = C3[["Asian Restaurant","Hotel","Indian Restaurant"]]==0.0

C3_res

,Asian Restaurant,Hotel,Indian Restaurant
1,True,True,True
2,True,True,False
3,True,False,True
4,True,False,True


In [98]:
C_r = C3_res["Asian Restaurant"]&C3_res["Hotel"]&C3_res["Indian Restaurant"]
C_r

1     True
2    False
3    False
4    False
dtype: bool

### Observation

From the above data analysis, we can predict that it would be a good decision if the restaurants are constructed in Cluster 3. Cluster 1 seems to be equipped with good hotels and restuarants nearby. On further analysis on Cluster 3, Cluster 2 and Cluster 1 we find that these Waterfalls have shortage of good hotels and restaurants nearby

- Courtallam
- Catherine Falls
- Thirparappu Falls
- Thirparappu
- Vaideki
- Siruvani Falls